In [2]:
import pandas as pd
import nltk as tm
from nltk.corpus import stopwords
import wordcloud
import matplotlib
from sklearn.feature_extraction.text import CountVectorizer

In [3]:
pwd

'c:\\Users\\Stenh\\OneDrive - Uppsala universitet\\Termin 4\\Examensarbete_analys'

In [9]:

df = pd.read_csv("data.csv", encoding="utf-8", delimiter = ",")

In [10]:
df.head()

,Tidstämpel,Kön,Ålder,Hur många invånare har orten där du bor?,Jag känner till GDPR.,Jag litar på att hemsidor hanterar mina personuppgifter på ett ansvarsfullt sätt.,"Jag väljer ""Godkänn alla cookies"".","Jag väljer ""Tillåt alla"".","Jag väljer ""Acceptera alla cookies"".","Jag väljer ""Acceptera alla cookies"". .1",...,Jag känner mig obekväm med att webbplatser samlar in information om mig med hjälp av cookies.,Jag känner oro över att information som samlas in via cookies kan användas på ett felaktigt sätt.,Jag känner mig obekväm med att webbplatser spårar mitt beteende med hjälp av cookies.,Jag tycker att det är viktigt att jag får välja vilka cookies som får lagras när jag besöker en webbplats.,Jag tycker att hemsidor bör förklara tydligt vad varje cookie används till.,Jag tycker att det är fel att samla in personuppgifter via cookies utan att informera användaren.,Jag tycker att företag bör be om tydligt samtycke innan de använder cookies som samlar in personlig data.,Jag tycker att företag inskränker på människors privatliv när de spårar aktivitet via cookies.,Jag tycker att det är rimligt att betala med personlig data istället för pengar för att använda tjänster online.,Hur var testet? Är det något vi skulle kunna förbättra eller göra tydligare?
0,2025/04/03 11:11:03 fm EET,Man,26-40,50-100 000,7,3,5,2,2,6,...,5,5,6,7,7,6,6,5,6,1
1,2025/04/03 11:59:11 fm EET,Annan,55+,200 000+,7,1,7,1,2,7,...,7,7,7,7,7,7,7,7,7,"Bra fina kak emojis med länkar, easter egg, br..."
2,2025/04/03 12:01:02 em EET,Man,26-40,50-100 000,7,6,7,7,4,7,...,5,5,4,6,7,5,6,5,5,NaN
3,2025/04/03 12:31:07 em EET,Man,18-25,10-50 000,7,1,1,1,1,1,...,7,7,7,7,7,7,7,7,1,Tydligt och bra forms. Ända jag vill tilläga ä...
4,2025/04/03 12:40:42 em EET,Kvinna,26-40,50-100 000,7,2,1,1,1,1,...,7,7,7,7,7,7,7,6,1,Intressant och bra test!


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 45 columns):
 #   Column                                                                                                              Non-Null Count  Dtype 
---  ------                                                                                                              --------------  ----- 
 0   Tidstämpel                                                                                                          5 non-null      object
 1   Kön                                                                                                                 5 non-null      object
 2   Ålder                                                                                                               5 non-null      object
 3   Hur många invånare har orten där du bor?                                                                            5 non-null      object
 4   Jag känner till GDPR.         

: 

In [11]:
df.select_dtypes(include=["number"]).corr()

,Jag känner till GDPR.,Jag litar på att hemsidor hanterar mina personuppgifter på ett ansvarsfullt sätt.,"Jag väljer ""Godkänn alla cookies"".","Jag väljer ""Tillåt alla"".","Jag väljer ""Acceptera alla cookies"".","Jag väljer ""Acceptera alla cookies"". .1","Jag väljer ""Tillåt alla"". .1",Mina val av cookies är olika beroende på vilken sida jag besöker.,Jag undviker vissa webbplatser på grund av hur de hanterar cookies.,Jag läser igenom informationen om cookies innan jag gör ett val.,...,Jag vill ha kontroll över vilka cookies som får samla in data om mig.,Jag känner mig obekväm med att webbplatser samlar in information om mig med hjälp av cookies.,Jag känner oro över att information som samlas in via cookies kan användas på ett felaktigt sätt.,Jag känner mig obekväm med att webbplatser spårar mitt beteende med hjälp av cookies.,Jag tycker att det är viktigt att jag får välja vilka cookies som får lagras när jag besöker en webbplats.,Jag tycker att hemsidor bör förklara tydligt vad varje cookie används till.,Jag tycker att det är fel att samla in personuppgifter via cookies utan att informera användaren.,Jag tycker att företag bör be om tydligt samtycke innan de använder cookies som samlar in personlig data.,Jag tycker att företag inskränker på människors privatliv när de spårar aktivitet via cookies.,Jag tycker att det är rimligt att betala med personlig data istället för pengar för att använda tjänster online.
Jag känner till GDPR.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Jag litar på att hemsidor hanterar mina personuppgifter på ett ansvarsfullt sätt.,NaN,1.000000,0.492871,9.616447e-01,8.859366e-01,0.492950,NaN,-4.189458e-02,-0.471576,-0.692679,...,-9.704950e-01,-8.364284e-01,-8.364284e-01,-9.801379e-01,-9.165786e-01,NaN,-9.704950e-01,-8.364284e-01,-8.439249e-01,2.557479e-01
"Jag väljer ""Godkänn alla cookies"".",NaN,0.492871,1.000000,5.562939e-01,8.075729e-01,0.989967,NaN,4.439457e-01,-0.012643,0.067651,...,-5.897678e-01,-5.417363e-01,-5.417363e-01,-5.815800e-01,-5.160468e-01,NaN,-5.897678e-01,-5.417363e-01,-3.296902e-01,9.325048e-01
"Jag väljer ""Tillåt alla"".",NaN,0.961645,0.556294,1.000000e+00,9.393364e-01,0.526746,NaN,-1.332593e-01,-0.213235,-0.481970,...,-9.432422e-01,-7.351470e-01,-7.351470e-01,-9.852941e-01,-9.861169e-01,NaN,-9.432422e-01,-7.351470e-01,-6.710944e-01,2.711631e-01
"Jag väljer ""Acceptera alla cookies"".",NaN,0.885937,0.807573,9.393364e-01,1.000000e+00,0.782461,NaN,8.866586e-02,-0.156556,-0.314140,...,-9.128709e-01,-7.453560e-01,-7.453560e-01,-9.393364e-01,-9.128709e-01,NaN,-9.128709e-01,-7.453560e-01,-6.123724e-01,5.773503e-01
"Jag väljer ""Acceptera alla cookies"". .1",NaN,0.492950,0.989967,5.267457e-01,7.824608e-01,1.000000,NaN,4.440164e-01,-0.116374,0.008193,...,-6.071429e-01,-6.123724e-01,-6.123724e-01,-5.757453e-01,-4.642857e-01,NaN,-6.071429e-01,-6.123724e-01,-3.992979e-01,9.599771e-01
"Jag väljer ""Tillåt alla"". .1",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Mina val av cookies är olika beroende på vilken sida jag besöker.,NaN,-0.041895,0.443946,-1.332593e-01,8.866586e-02,0.444016,NaN,1.000000e+00,-0.258190,-0.066848,...,9.712859e-02,3.965258e-02,3.965258e-02,1.166019e-01,1.456929e-01,NaN,9.712859e-02,3.965258e-02,-1.085931e-01,5.375082e-01
Jag undviker vissa webbplatser på grund av hur de hanterar cookies.,NaN,-0.471576,-0.012643,-2.132353e-01,-1.565561e-01,-0.116374,NaN,-2.581899e-01,1.000000,0.895087,...,4.501838e-01,6.651330e-01,6.651330e-01,3.382353e-01,8.574929e-02,NaN,4.501838e-01,6.651330e-01,8.628356e-01,-1.016862e-01
Jag läser igenom informationen om cookies innan jag gör ett val.,NaN,-0.692679,0.067651,-4.819698e-01,-3.141404e-01,0.008193,NaN,-6.684848e-02,0.895087,1.000000,...,6.022163e-01,6.556101e-01,6.556101e-01,5.508226e-01,4.014775e-01,NaN,6.022163e-01,6.556101e-01,8.977311e-01,1.360268e-01
